## Bayes Theorem

### $P(y|X) = \frac{P(X|y)P(y)}{P(X)}$

### $X = (x_1, x_2. x_3, \ldots, x_n)$

### Assume that all features are mutually independent. Use chain rule for $P(X|y)$

### $P(y|X)=\frac{P(x_1|y)\cdot P(x_1|y)\cdot \ldots \cdot P(X_n|y)\cdot P(y)}{P(x)}$

$P(y|X)$ is the posterior probability

$P(X|y)$ is the class conditional probability

$P(y)$ is the prior probability of y

### Select the class with highest probability

### $y = argmax_yP(y|X)=argmax_y=\frac{P(x_1|y)\cdot P(x_1|y)\cdot \ldots \cdot P(X_n|y)\cdot P(y)}{P(x)}$

we don't need $P(X)$ and we apply log to combat floating point problems

### $y = argmax_yP(y|X)=argmax_ylog(P(x_1|y))+log(P(x_2|y))+ \ldots + log(P(x_n|y)) + log(P(y)))$

### Class conditional probability is modelled with a gaussian distribution

### $P(x_i|y)=\frac{1}{\sqrt{2\pi \sigma^{2}_{y}}} exp(- \frac{(x_i-\mu_y)^2}{2\sigma^{2}_{y}})$

In [3]:
import numpy as np


class NaiveBayes:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)

        for idx, c in enumerate(self._classes):
            X_c = X[y == c]
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            class_conditional = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + class_conditional
            posteriors.append(posterior)

        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)]

    def _pdf(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


## Testing

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

X, y = datasets.make_classification(
    n_samples=1000, n_features=10, n_classes=2, random_state=123
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123
)

nb = NaiveBayes()
nb.fit(X_train, y_train)
predictions = nb.predict(X_test)

print("Naive Bayes classification accuracy", accuracy(y_test, predictions))

Naive Bayes classification accuracy 0.965


Source: https://www.youtube.com/watch?v=rLOyrWV8gmA